# Introduction to the Planner

The Planner is one of the fundamental concepts of the Semantic Kernel.

It makes use of the collection of native and semantic functions that have been registered to the kernel and using AI, will formulate a plan to execute the given ask.

From our own testing, planner works best with more powerful models like `gpt4` but sometimes you might get working plans with cheaper models like `gpt-35-turbo`. We encourage you to implement your own versions of the planner and use different models that fit your user needs.

Read more about planner [here](https://aka.ms/sk/concepts/planner)


In [ ]:
!python -m pip install -U semantic-kernel==0.9.6b1

In [4]:
from services import Service

# Select a service to use for this notebook (available services: OpenAI, AzureOpenAI, HuggingFace)
selectedService = Service.AzureOpenAI

In [5]:
from semantic_kernel import Kernel  # noqa: F401
from semantic_kernel.connectors.ai.open_ai import (  # noqa: F401
    AzureChatCompletion,
    OpenAIChatCompletion,
    OpenAIChatPromptExecutionSettings,
)
from semantic_kernel.contents import ChatHistory  # noqa: F401
from semantic_kernel.functions import KernelArguments  # noqa: F401
from semantic_kernel.prompt_template import InputVariable  # noqa: F401
from semantic_kernel.utils.settings import (  # noqa: F401
    azure_openai_settings_from_dot_env,
    openai_settings_from_dot_env,
)

In [7]:
kernel = Kernel()
selectedService = Service.OpenAI
service_id = None
if selectedService == Service.OpenAI:
    api_key, org_id = openai_settings_from_dot_env()
    service_id = "default"
    kernel.add_service(
        OpenAIChatCompletion(service_id=service_id, ai_model_id="gpt-3.5-turbo-1106", api_key=api_key, org_id=org_id),
    )
elif selectedService == Service.AzureOpenAI:
    deployment, api_key, endpoint = azure_openai_settings_from_dot_env()
    service_id = "default"
    kernel.add_service(
        AzureChatCompletion(service_id=service_id, deployment_name=deployment, endpoint=endpoint, api_key=api_key),
    )

## It all begins with an ask


In [9]:
ask = """
Tomorrow is Valentine's day. I need to come up with a few date ideas. She speaks French so write it in French.
Convert the text to uppercase"""

### Providing plugins to the planner

The planner needs to know what plugins are available to it. Here we'll give it access to the `SummarizePlugin` and `WriterPlugin` we have defined on disk. This will include many semantic functions, of which the planner will intelligently choose a subset.

You can also include native functions as well. Here we'll add the TextPlugin.


In [10]:
from semantic_kernel.core_plugins import TextPlugin

plugins_directory = "../../samples/plugins/"
summarize_plugin = kernel.add_plugin(parent_directory=plugins_directory, plugin_name="SummarizePlugin")
writer_plugin = kernel.add_plugin(parent_directory=plugins_directory, plugin_name="WriterPlugin")
text_plugin = kernel.add_plugin(TextPlugin(), "TextPlugin")

Define your ASK. What do you want the Kernel to do?


# Basic Planner


Let's start by taking a look at a basic planner. The `BasicPlanner` produces a JSON-based plan that aims to solve the provided ask sequentially and evaluated in order.


In [11]:
from semantic_kernel.planners import BasicPlanner

planner = BasicPlanner(service_id)

In [12]:
basic_plan = await planner.create_plan(ask, kernel)

In [15]:
print(basic_plan.generated_plan)

{
    "input": "Valentine's Day Date Ideas",
    "subtasks": [
        {"function": "WriterPlugin.Brainstorm"},
        {"function": "WriterPlugin.EmailGen"},
        {"function": "WriterPlugin.Translate", "args": {"language": "French"}},
        {"function": "TextPlugin.uppercase"}
    ]
}


You can see that the Planner took my ask and converted it into an JSON-based plan detailing how the AI would go about solving this task, making use of the plugins that the Kernel has available to it.

As you can see in the above plan, the AI has determined which functions to call in order to fulfill the user ask. The output of each step of the plan becomes the input to the next function.


Let's also define an inline plugin and have it be available to the Planner. Be sure to give it a function name and plugin name.


Let's update our ask using this new plugin


In [16]:
from semantic_kernel.functions import KernelFunctionFromPrompt

kernel = Kernel()
service_id = "default"
if selectedService == Service.OpenAI:
    api_key, org_id = openai_settings_from_dot_env()
    kernel.add_service(
        OpenAIChatCompletion(service_id=service_id, ai_model_id="gpt-3.5-turbo-1106", api_key=api_key, org_id=org_id),
    )
elif selectedService == Service.AzureOpenAI:
    deployment, api_key, endpoint = azure_openai_settings_from_dot_env()
    kernel.add_service(
        AzureChatCompletion(service_id=service_id, deployment_name=deployment, endpoint=endpoint, api_key=api_key),
    )

plugins_directory = "../../samples/plugins/"
summarize_plugin = kernel.add_plugin(parent_directory=plugins_directory, plugin_name="SummarizePlugin")
writer_plugin = kernel.add_plugin(parent_directory=plugins_directory, plugin_name="WriterPlugin")
text_plugin = kernel.add_plugin(TextPlugin(), "TextPlugin")

shakespeare_func = KernelFunctionFromPrompt(
    function_name="Shakespeare",
    plugin_name="WriterPlugin",
    prompt="""
{{$input}}

Rewrite the above in the style of Shakespeare.
""",
    prompt_execution_settings=OpenAIChatPromptExecutionSettings(
        service_id=service_id,
        max_tokens=2000,
        temperature=0.8,
    ),
)
kernel.add_function("WriterPlugin", shakespeare_func)

for plugin in kernel.plugins.values():
    for function in plugin:
        print(f"Plugin: {plugin.name}, Function: {function.name}")

Plugin: SummarizePlugin, Function: Topics
Plugin: SummarizePlugin, Function: MakeAbstractReadable
Plugin: SummarizePlugin, Function: Notegen
Plugin: SummarizePlugin, Function: Summarize
Plugin: WriterPlugin, Function: NovelChapterWithNotes
Plugin: WriterPlugin, Function: AcronymReverse
Plugin: WriterPlugin, Function: NovelOutline
Plugin: WriterPlugin, Function: AcronymGenerator
Plugin: WriterPlugin, Function: Rewrite
Plugin: WriterPlugin, Function: EmailTo
Plugin: WriterPlugin, Function: NovelChapter
Plugin: WriterPlugin, Function: Translate
Plugin: WriterPlugin, Function: EnglishImprover
Plugin: WriterPlugin, Function: ShortPoem
Plugin: WriterPlugin, Function: Acronym
Plugin: WriterPlugin, Function: TwoSentenceSummary
Plugin: WriterPlugin, Function: Brainstorm
Plugin: WriterPlugin, Function: TellMeMore
Plugin: WriterPlugin, Function: StoryGen
Plugin: WriterPlugin, Function: EmailGen
Plugin: WriterPlugin, Function: Shakespeare
Plugin: TextPlugin, Function: lowercase
Plugin: TextPlugin,

In [18]:
planner = BasicPlanner(service_id)

ask = """
Tomorrow is Valentine's day. I need to come up with a few short poems.
She likes Shakespeare so write using his style. She speaks French so write it in French.
Convert the text to uppercase."""

new_plan = await planner.create_plan(goal=ask, kernel=kernel)

In [19]:
print(new_plan.generated_plan)

{
  "input": "Valentine's Day Short Poems",
  "subtasks": [
    {"function": "WriterPlugin.ShortPoem"},
    {"function": "WriterPlugin.Shakespeare"},
    {"function": "WriterPlugin.Translate", "args": {"language": "French"}},
    {"function": "TextPlugin.uppercase"}
  ]
}


### Executing the plan


Now that we have a plan, let's try to execute it! The Planner has a function called `execute_plan`.


In [20]:
results = await planner.execute_plan(new_plan, kernel)

In [21]:
print(results)

ROSES D'UN ROUGE CRAMOISI, VIOLETTES D'AZUR
LA SAINT-VALENTIN APPROCHE, QUE DEVRIONS-NOUS ACHETER?
ACHETER DES SUCRERIES, ÉCRIRE UNE LETTRE D'AMOUR
OU SIMPLEMENT RESTER À L'INTÉRIEUR ET ENSEMBLE, REGARDER EN RAFALE
EN TOUT CAS, C'EST UN JOUR POUR L'AMOUR ET LA JOIE
RÉJOUISSONS-NOUS ET VIVONS HEUREUX DÉSORMAIS


# The Plan Object Model


To build more advanced planners, we need to introduce a proper Plan object that can contain all the necessary state and information needed for high quality plans.

To see what that object model is, look at (https://github.com/microsoft/semantic-kernel/blob/main/python/semantic_kernel/planners/plan.py)


# Sequential Planner


The sequential planner is an XML-based step-by-step planner. You can see the prompt used for it here (https://github.com/microsoft/semantic-kernel/blob/main/python/semantic_kernel/planners/sequential_planner/Plugins/SequentialPlanning/skprompt.txt)


In [22]:
from semantic_kernel.planners import SequentialPlanner

planner = SequentialPlanner(kernel, service_id)

In [23]:
sequential_plan = await planner.create_plan(goal=ask)

To see the steps that the Sequential Planner will take, we can iterate over them and print their descriptions


In [24]:
for step in sequential_plan._steps:
    print(step.description, ":", step._state.__dict__)

Turn a scenario into a short and entertaining poem. : {'execution_settings': {}}
None : {'execution_settings': {}}
Translate the input into a language of your choice : {'execution_settings': {}}
Convert a string to uppercase. : {'execution_settings': {}}


Let's ask the sequential planner to execute the plan.


In [25]:
result = await sequential_plan.invoke(kernel)

In [26]:
print(result)

ATTENTION! ROSES ROUGES, ET VIOLETTES BLEUES
LE JOUR DE LA SAINT-VALENTIN, QUE FERAS-TU?
ACHÈTERAS-TU DES CHOCOLATS OU UN GRAND BOUQUET?
OU BIEN, RESTERAS-TU À LA MAISON ET TE LAISSERAS ALLER À MANGER DE LA PIZZA TOUTE LA JOURNÉE?
PEU IMPORTE LE CHEMIN QUE TU CHOISIS, RÉJOUIS-TOI DANS LA JOIE
ET GARDE À L'ESPRIT, L'AMOUR EST POUR TOUS SUR CETTE TERRE!


# Action Planner


The action planner takes in a list of functions and the goal, and outputs a **single** function to use that is appropriate to meet that goal.


In [27]:
from semantic_kernel.planners import ActionPlanner

planner = ActionPlanner(kernel, service_id)

Let's add more plugins to the kernel


In [28]:
from semantic_kernel.core_plugins import MathPlugin, TextPlugin, TimePlugin

kernel.add_plugin(MathPlugin(), "math")
kernel.add_plugin(TimePlugin(), "time")
kernel.add_plugin(TextPlugin(), "text")

KernelPlugin(name='text', description=None, functions={'lowercase': KernelFunctionFromMethod(metadata=KernelFunctionMetadata(name='lowercase', plugin_name='text', description='Convert a string to lowercase.', parameters=[KernelParameterMetadata(name='input', description='', default_value=None, type_='str', is_required=True, type_object=<class 'str'>)], is_prompt=False, is_asynchronous=False, return_parameter=KernelParameterMetadata(name='return', description='', default_value=None, type_='str', is_required=True, type_object=None)), method=<bound method TextPlugin.lowercase of TextPlugin()>, stream_method=None), 'trim': KernelFunctionFromMethod(metadata=KernelFunctionMetadata(name='trim', plugin_name='text', description='Trim whitespace from the start and end of a string.', parameters=[KernelParameterMetadata(name='input', description='', default_value=None, type_='str', is_required=True, type_object=<class 'str'>)], is_prompt=False, is_asynchronous=False, return_parameter=KernelParamet

In [29]:
ask = "What is the sum of 110 and 990?"

In [30]:
plan = await planner.create_plan(goal=ask)

WriterPlugin.Shakespeare is missing a description
PlannerPlugin.CreatePlan is missing a description
SequentialPlanner_Excluded.SequentialPlanner_Excluded is missing a description
PlannerPlugin.CreatePlan is missing a description
SequentialPlanner_Excluded.SequentialPlanner_Excluded is missing a description


In [31]:
result = await plan.invoke(kernel)

In [32]:
print(result)

1100


# Stepwise Planner


Stepwise Planner is based off the paper from MRKL (Modular Reasoning, Knowledge and Language) and is similar to other papers like ReACT (Reasoning and Acting in Language Models). At the core, the stepwise planner allows for the AI to form "thoughts" and "observations" and execute actions based off those to achieve a user's goal. This continues until all required functions are complete and a final output is generated.

See a video walkthrough of Stepwise Planner [here.](https://youtu.be/DG_Ge1v0c4Q?si=T1CHaAm1vV0mWRHu)


Let's create a Bing Search native plugin that we can pass in to the Kernel.

Make sure you have a Bing Search API key in your `.env` file

(https://www.microsoft.com/en-us/bing/apis/bing-web-search-api)


In [33]:
from semantic_kernel.connectors.search_engine import BingConnector
from semantic_kernel.core_plugins import WebSearchEnginePlugin
from semantic_kernel.utils.settings import bing_search_settings_from_dot_env

BING_API_KEY = bing_search_settings_from_dot_env()
connector = BingConnector(BING_API_KEY)
kernel.add_plugin(WebSearchEnginePlugin(connector), plugin_name="WebSearch")

KernelPlugin(name='WebSearch', description=None, functions={'search': KernelFunctionFromMethod(metadata=KernelFunctionMetadata(name='search', plugin_name='WebSearch', description='Performs a web search for a given query', parameters=[KernelParameterMetadata(name='query', description='The search query', default_value=None, type_='str', is_required=True, type_object=<class 'str'>), KernelParameterMetadata(name='num_results', description='The number of search results to return', default_value=1, type_='int', is_required=False, type_object=<class 'int'>), KernelParameterMetadata(name='offset', description='The number of search results to skip', default_value=0, type_='int', is_required=False, type_object=<class 'int'>)], is_prompt=False, is_asynchronous=True, return_parameter=KernelParameterMetadata(name='return', description='', default_value=None, type_='str', is_required=True, type_object=None)), method=<bound method WebSearchEnginePlugin.search of <semantic_kernel.core_plugins.web_sear

Let's also add a couple more plugins


In [34]:
from semantic_kernel.core_plugins import MathPlugin, TimePlugin

kernel.add_plugin(TimePlugin(), "time")
kernel.add_plugin(MathPlugin(), "math")

KernelPlugin(name='math', description=None, functions={'Add': KernelFunctionFromMethod(metadata=KernelFunctionMetadata(name='Add', plugin_name='math', description='Returns the Addition result of the values provided.', parameters=[KernelParameterMetadata(name='input', description='the first number to add', default_value=None, type_='int', is_required=True, type_object=<class 'int'>), KernelParameterMetadata(name='amount', description='the second number to add', default_value=None, type_='int', is_required=True, type_object=<class 'int'>)], is_prompt=False, is_asynchronous=False, return_parameter=KernelParameterMetadata(name='return', description='the output is a number', default_value=None, type_='int', is_required=True, type_object=None)), method=<bound method MathPlugin.add of <semantic_kernel.core_plugins.math_plugin.MathPlugin object at 0x783ee4e00760>>, stream_method=None), 'Subtract': KernelFunctionFromMethod(metadata=KernelFunctionMetadata(name='Subtract', plugin_name='math', des

In [35]:
from semantic_kernel.planners import StepwisePlanner, StepwisePlannerConfig

planner = StepwisePlanner(kernel, StepwisePlannerConfig(max_iterations=10, min_iteration_time_ms=1000))

Now let's do a more complicated ask that will require planner to make a call to Bing to get the latest information.


In [36]:
ask = """How many total championships combined do the top 5 teams in the NBA have? And which teams are they?"""

plan = planner.create_plan(goal=ask)

In [37]:
result = await plan.invoke(kernel)

In [38]:
print(result)

The top 5 teams in the NBA with the most championships are the Celtics and Lakers, both tied with 17 championships each. The total number of championships combined for the top 5 teams is 85.


Let's see the steps that the AI took to get to the answer.


In [39]:
for index, step in enumerate(plan._steps):
    print("Step:", index)
    print("Description:", step.description)
    print("Function:", step.plugin_name + "." + step._function.name)
    print(f"  Output: {','.join(str(res) for res in result.metadata['results'])}")

Step: 0
Description: Execute a plan
Function: StepwisePlanner.ExecutePlan
  Output: The top 5 teams in the NBA with the most championships are the Celtics and Lakers, both tied with 17 championships each. The total number of championships combined for the top 5 teams is 85.
